In [ ]:
# Calculating Speed per day, per vehicle, per region, 

In [ ]:
import pyspark.sql.functions as F

# calculating speed per vehicle
for day in range(1,32):
    traces = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/13-speed-calculation-filtered-columns/MO_1510"+str(day)+"/")
    df_speed = traces.groupby("id_avl","line_id").agg(F.avg("speed").alias("avg_speed"),F.stddev("speed").alias("speed_stddev"))
    df_speed.write.parquet("s3://mobility-traces-sp/metrics-calculation/speed-calculation/speed-per-vehicle/MO_1510"+str(day)+"/")
    
    

In [ ]:
# installing required packages for this notebook session
sc.install_pypi_package("boto3")

In [ ]:
# avg speed per day

import pyspark.sql.functions as F

# statistics speed per vehicle
csv_out_vehicle = "day,n_vehicles,n_lines,n_vehicles_lines,avg_speed,avg_speed_min,avg_speed_max,avg_speed_stddev,avg_speed_quantile_6.25,avg_speed_quantile_12.5,avg_speed_quantile_25,avg_speed_quantile_50,avg_speed_quantile_75\n"

for day in range(1,32):
    # speed by vehicle per line
    speeds_per_vehicle = spark.read.parquet("s3://mobility-traces-sp/metrics-calculation/speed-calculation/speed-per-vehicle/MO_1510"+str(day)+"/")
    
    sizes = speeds_per_vehicle.select(F.countDistinct("id_avl").alias("vehicles"),
                                      F.countDistinct("line_id").alias("lines"),
                                      F.countDistinct("id_avl","line_id").alias("vehicles_lines")
                                    ).collect()
    
    stats = speeds_per_vehicle.agg(F.mean('avg_speed').alias('mean'),
                       F.min('avg_speed').alias('min'),
                       F.max('avg_speed').alias('max'),
                       F.stddev('avg_speed').alias("stddev")).collect()
    
    quantiles = speeds_per_vehicle.approxQuantile("avg_speed", [0.0625,0.125,0.25,0.5,0.75], 0.0001)
    
    csv_out_vehicle += "{},{},{},{},{},{},{},{},{},{},{},{},{}\n"\
        .format("MO_1510"+str(day),
                sizes[0]["vehicles"],sizes[0]["lines"],sizes[0]["vehicles_lines"],
                stats[0]["mean"],stats[0]["min"],stats[0]["max"],stats[0]["stddev"],
                quantiles[0],quantiles[1],quantiles[2],quantiles[3],quantiles[4]
        )

import boto3   
s3 = boto3.client('s3')

s3.put_object(Body=bytes(csv_out_vehicle,"utf-8"), Bucket='mobility-traces-sp', Key='metrics-calculation/speed-calculation/statistics/1-speed_per_vehicle_stats.csv')


In [ ]:
# Calculating speed by region per day
# 1- Calculating the avg speed per bus per region --> groupby id_avl,line_id,region
# 2- Calculating the avg speed per region per day --> groupby region
import pyspark.sql.functions as F

# calculating speed per region per day
for day in range(1,32):
    # Reading traces
    traces = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/13-speed-calculation-filtered-columns/MO_1510"+str(day)+"/")
    
    # Calculating speed by vehicle per region
    df_speed_region_per_vehicle = traces.groupby("id_avl","line_id","region").agg(F.avg("speed").alias("avg_speed"),F.stddev("speed").alias("speed_stddev"))
    df_speed_region_per_vehicle.write.parquet("s3://mobility-traces-sp/metrics-calculation/speed-calculation/speed-per-vehicle-per-region/MO_1510"+str(day)+"/")
    
    # Calculating speed by region per day
    df_speed_region_day = df_speed_region_per_vehicle.groupby("region").agg(F.avg("avg_speed").alias("avg_speed"),F.stddev("avg_speed").alias("speed_stddev"))
    df_speed_region_day.write.parquet("s3://mobility-traces-sp/metrics-calculation/speed-calculation/speed-per-region-per-day/MO_1510"+str(day)+"/")


In [ ]:
# installing required packages for this notebook session
sc.install_pypi_package("boto3")

In [ ]:
import pyspark.sql.functions as F

# statistics speed per region per day
csv_out_region = "day,avg_speed,avg_speed_min,avg_speed_max,avg_speed_stddev,avg_speed_quantile_6.25,avg_speed_quantile_12.5,avg_speed_quantile_25,avg_speed_quantile_50,avg_speed_quantile_75\n"

for day in range(1,32):
    
    # speed by region per day
    speeds_per_region = spark.read.parquet("s3://mobility-traces-sp/metrics-calculation/speed-calculation/speed-per-region-per-day/MO_1510"+str(day)+"/") 
    
    stats = speeds_per_region.agg(F.mean('avg_speed').alias('mean'),
                       F.min('avg_speed').alias('min'),
                       F.max('avg_speed').alias('max'),
                       F.stddev('avg_speed').alias("stddev")).collect()
    
    quantiles = speeds_per_region.approxQuantile("avg_speed", [0.0625,0.125,0.25,0.5,0.75], 0.0001)
    
    csv_out_region += "{},{},{},{},{},{},{},{},{},{}\n"\
        .format("MO_1510"+str(day),
                stats[0]["mean"],stats[0]["min"],stats[0]["max"],stats[0]["stddev"],
                quantiles[0],quantiles[1],quantiles[2],quantiles[3],quantiles[4]
        )

import boto3   
s3 = boto3.client('s3')

s3.put_object(Body=bytes(csv_out_region,"utf-8"), Bucket='mobility-traces-sp', Key='metrics-calculation/speed-calculation/statistics/2-speed_per_region_stats.csv')


In [10]:
# Calculating speed by hour per day
# 1- Calculating the avg speed per bus per hour --> groupby id_avl,line_id,hour
# 2- Calculating the avg speed per hour per day --> groupby hour
import pyspark.sql.functions as F

# calculating speed per hour per day
for day in range(1,32):
    # Reading traces
    traces = spark.read.parquet("s3://mobility-traces-sp/processed-data-avl-date/13-speed-calculation-filtered-columns/MO_1510"+str(day)+"/")
    
    # Calculating speed by vehicle per hour
    df_speed_hour_per_vehicle = traces.groupby("id_avl","line_id","hour_avl").agg(F.avg("speed").alias("avg_speed"),F.stddev("speed").alias("speed_stddev"))
    df_speed_hour_per_vehicle.write.parquet("s3://mobility-traces-sp/metrics-calculation/speed-calculation/speed-per-vehicle-per-hour/MO_1510"+str(day)+"/")
    
    # Calculating speed by hour per day
    df_speed_hour_day = df_speed_hour_per_vehicle.groupby("hour_avl").agg(F.avg("avg_speed").alias("avg_speed"),F.stddev("avg_speed").alias("speed_stddev"))
    df_speed_hour_day.write.parquet("s3://mobility-traces-sp/metrics-calculation/speed-calculation/speed-per-hour-per-day/MO_1510"+str(day)+"/")


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Calculating speed by hour and region per day
# 1- Calculating the avg speed per bus per hour and region --> groupby id_avl,line_id,hour,region
# 2- Calculating the avg speed per hour and region per day --> groupby hour, region
import pyspark.sql.functions as F

for day in range(1,32):
    # Reading traces
    traces = spark.read.parquet(f"s3://mobility-traces-sp/processed-data-avl-date/13-speed-calculation-filtered-columns/MO_1510{day}/")    
   
    # Calculating speed by vehicle per hour and region
    df_speed_hour_region = traces.groupby("id_avl","line_id","hour_avl","region").agg(F.avg("speed").alias("avg_speed"),F.stddev("speed").alias("speed_stddev"))
    df_speed_hour_region.write.parquet("s3://mobility-traces-sp/metrics-calculation/speed-calculation/speed-per-vehicle-hour-region-aggregration/MO_1510"+str(day)+"/")
    
    # Calculating speed by hour and region per day
    df_speed_hour_region_day = df_speed_hour_region.groupby("hour_avl","region").agg(F.avg("avg_speed").alias("avg_speed"),F.stddev("avg_speed").alias("speed_stddev"))
    df_speed_hour_region_day.write.parquet("s3://mobility-traces-sp/metrics-calculation/speed-calculation/speed-per-hour-per-region-per-day/MO_1510"+str(day)+"/")
